In [1]:
!pip install llama-index python-dotenv "httpx==0.27.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [4]:
import os
import nest_asyncio

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

os.environ["OPENAI_API_KEY"] = "sk"
nest_asyncio.apply() # 중첩 실행 허용

In [8]:
from llama_index.llms.openai import OpenAI

# LLM 설정
llm = OpenAI(model="gpt-4o")  # 원하는 LLM 모델 설정

In [10]:
pdf_file = SimpleDirectoryReader(input_dir="/content/drive/MyDrive/store/").load_data()

In [11]:
vector_query_engine = VectorStoreIndex.from_documents(  # 문서를 벡터로 변환하여 저장
    pdf_file,
    use_async=True,
).as_query_engine() # VectorStoreIndex을 기반으로 질의 응답(Query Engine) 기능을 활성화

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Q&A bot",
            description="당신은 사용자 질문에 답변하는 봇입니다",
        ),
    ),
] # 사용자 질문에 대한 답변 생성

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=llm,  # 추가된 LLM 사용
    use_async=True,
) # 복잡한 질문을 하위 질문(Sub-Question)으로 분리하여 처리하고 결과를 조합

In [13]:
response = query_engine.query(
    "한국형 스마트팜을 구축하기위해 필요한 기술은?"
)
print(response)

Generated 6 sub questions.
[Q&A bot] Q: 한국형 스마트팜을 구축하기 위해 필요한 주요 기술은 무엇인가요?
[Q&A bot] Q: 스마트팜 구축 시 고려해야 할 한국의 기후 및 환경적 요인은 무엇인가요?
[Q&A bot] Q: 한국형 스마트팜에 적합한 자동화 기술은 무엇인가요?
[Q&A bot] Q: 스마트팜 운영에 필요한 데이터 분석 기술은 무엇인가요?
[Q&A bot] Q: 한국형 스마트팜에서 활용할 수 있는 IoT 기술은 무엇인가요?
[Q&A bot] Q: 스마트팜의 에너지 효율성을 높이기 위한 기술은 무엇인가요?
[Q&A bot] A: 빅데이터 분석 기술
[Q&A bot] A: 에너지 효율성을 높이기 위한 기술은 에너지 최적화 및 로봇 자동화 등 스마트팜 통합시스템을 구현하는 것입니다.
[Q&A bot] A: When establishing a smart farm in Korea, it is important to consider the country's aging agricultural population, high labor intensity, difficulty in attracting young farmers, and the need for efficient agricultural practices to enhance productivity and reduce labor. Additionally, factors such as the optimal management of crop growth environments, labor reduction, and productivity improvement through technologies like IoT, big data, AI, and robotics should be taken into account to address the challenges in the agricultural sector.
[Q&A bot] A: 사물인터넷(IoT) 기술을 활용하여 한국형 스마트팜